In [1]:


# This file tests Algorithm 10.

In [2]:
exec(open("./funcs/tool_funcs.py").read())



In [3]:
def dijkstra_show_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = current_distance + weight

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [4]:

def dijkstra_cal_previous_nodes(adj_matrix, start):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue
 
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  
                distance = current_distance + weight
 
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))
 
    return previous_nodes



In [5]:
def dijkstra_show_path_from_previous_nodes(previous_nodes, start, end): 
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()
    return path if path[0] == start else []

In [6]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [7]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [8]:
def cal_candidate_node_list(i, j, X_APSP_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if X_APSP_matrix[i,j] < X_APSP_matrix[i,t] + X_APSP_matrix[t,j]:
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [9]:
def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

In [10]:
# All shortest paths calculated by Algorithm 8 and 9 may be incomplete.
# However, this does not affect the correctness of Algorithm 10.

def cal_all_paths_warm_start(i, j, X_APSP_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, X_APSP_matrix, X_distance_matrix)
 
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    
    #Using this temp_list can save some time. 
    temp_list = [q for q in range(K) if q not in path]
    
    # temp_list = [q for q in range(1, K - 1)]
    
    previous_nodes = dijkstra_cal_previous_nodes(small_matrix, 0)

    for m in temp_list:
        # temp_path1 = dijkstra_show_path(small_matrix, 0, m)
        
        # There is no need to call dijkstra_show_path for each node m. 
        temp_path1 = dijkstra_show_path_from_previous_nodes(previous_nodes, 0, m)
 
        temp_path2 = dijkstra_show_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [11]:
def cal_key_node_list_all(X_distance_matrix, X_APSP_matrix):
    
    N = len(X_distance_matrix)
    
    key_node_list_all = []

    for i in range(N):
        for j in range(N):
            if i == j:
                key_node_list_all.append([])
            else:
                all_paths_list = cal_all_paths_warm_start(i, j, X_APSP_matrix, X_distance_matrix)
                key_node_list = set(all_paths_list[0])
                for m, path_m in enumerate(all_paths_list):
                    if m > 0:
                        aaa = set(path_m)
                        key_node_list = key_node_list.intersection(aaa)
                key_node_list = list(key_node_list)
                key_node_list.remove(i)
                key_node_list.remove(j)
                key_node_list_all.append(key_node_list)
    
    return key_node_list_all     


In [12]:
def cal_need_update_list_key_node_list(remove_node, key_node_list_all, N):
 
    remaining_list = [i for i in np.arange(N) if i != remove_node]
    need_update_list = [[] for i in range(N)]
    how_many_counter = np.zeros(N)
    for i in remaining_list:
        for j in remaining_list:
            if i != j:
                key_node_list = key_node_list_all[i*N + j]
                if remove_node in key_node_list:
                    need_update_list[i].append(j)
                    how_many_counter[i] += 1
    
    how_many = np.sum(how_many_counter > 0)
    
    return need_update_list, how_many

In [13]:
def cal_new_APSP_matrix_warm_key_node_list(X_distance_matrix, X_APSP_matrix, remove_node, key_node_list_all):
    N = len(X_distance_matrix)
    X_APSP_matrix_new = X_APSP_matrix.copy()
    remaining_list = [i for i in np.arange(N) if i != remove_node]
    

    need_update_list, how_many = cal_need_update_list_key_node_list(remove_node, key_node_list_all, N)
    
    Cost = np.round(how_many/(N - 1), 3)
    
    print(f"The cost for re-caculating the APSP matrix is: {Cost}.")

    X_distance_matrix_new = generate_new_distance_matrix(X_distance_matrix, remove_node)
    
    total = 0
    num_equal = 0
 
    for i in remaining_list:  
        if len(need_update_list[i]) > 0:
            temp = dijkstra_one_to_all_early_stop(X_distance_matrix_new, i, need_update_list[i], remove_node)
            for j in need_update_list[i]:
                X_APSP_matrix_new[i,j] = temp[j]
                total += 1
                if X_APSP_matrix[i,j] == X_APSP_matrix_new[i,j]:
                    num_equal += 1
 
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_APSP_matrix_new[i,j] = np.inf

    X_APSP_matrix_new[remove_node,remove_node] = 0
    
    print(f"Total number of recalculation: ", total)
    print(f"Number of equal to previous value: ", num_equal)
    
    return X_APSP_matrix_new 

In [14]:
def generate_new_distance_matrix(X_distance_matrix, remove_node):
    X_distance_matrix_new = X_distance_matrix.copy()
 
    N = len(X_distance_matrix)
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_distance_matrix_new[i,j] = np.inf

    X_distance_matrix_new[remove_node,remove_node] = 0  
    
    return X_distance_matrix_new

In [15]:
def dijkstra_one_to_all_early_stop(distance_matrix, src, need_up_i, remove_node):
    
    N = len(distance_matrix)
    dist = [np.inf] * N
    dist[src] = 0
    visited = [False] * N

    # Priority queue: (distance, node)
    pq = [(0, src)]
    
    kk = len(need_up_i)
    counter = 0
    visited[remove_node] = True

    while pq:
        current_dist, u = heapq.heappop(pq)

        if visited[u]:
            continue
        visited[u] = True
        
        if u in need_up_i:
            counter += 1
            if counter == kk:
                return np.array(dist)
 

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] > 0:
                new_dist = current_dist + distance_matrix[u][v]
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))

    return np.array(dist)

In [16]:
# N = 1000

# X_distance_matrix = create_distance_matrix(N)



In [17]:


X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")

X_APSP_matrix = floyd_warshall(X_distance_matrix)
 

In [18]:
start = time.time()
key_node_list_all = cal_key_node_list_all(X_distance_matrix, X_APSP_matrix)
end = time.time()


time_used_key_node_list_all = end - start
time_used_key_node_list_all = np.round(time_used_key_node_list_all, 3)
print(f"Time used for calculating key_node_list_all: {time_used_key_node_list_all}s" )

Time used for calculating key_node_list_all: 1.174s


In [19]:
remove_node = 3 

X_distance_matrix_new = generate_new_distance_matrix(X_distance_matrix, remove_node)

In [20]:

start = time.time()
X_APSP_matrix_new_warm = cal_new_APSP_matrix_warm_key_node_list(X_distance_matrix, X_APSP_matrix, remove_node, key_node_list_all)
end = time.time()


time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

print(f"time_used1_warm: {time_used1_warm}s" )


The cost for re-caculating the APSP matrix is: 0.434.
Total number of recalculation:  269
Number of equal to previous value:  0
time_used1_warm: 0.138s


In [21]:
start = time.time()
X_APSP_matrix_new_floyd_warshall = floyd_warshall(X_distance_matrix_new)
end = time.time()


time_used2_floyd = end - start
time_used2_floyd = np.round(time_used2_floyd, 3)
print(f"time_used2_floyd: {time_used2_floyd}s" )

time_used2_floyd: 1.02s


In [22]:

# Although it can be calculated in advance of knowing which node is to be removed, 
# the key_node_list_all in Algorithm 10 is very expensive to calculate, 
# the time complexity is at least  O(n^3) .

print(time_used_key_node_list_all)

1.174


In [23]:
 
print(f"Ratio: {np.round(time_used1_warm/time_used2_floyd, 2)}")

Ratio: 0.14


In [24]:
print(np.allclose(X_APSP_matrix_new_warm, X_APSP_matrix_new_floyd_warshall))


True


In [25]:
print(X_APSP_matrix_new_warm[0,:10])

print(X_APSP_matrix_new_warm.shape)

[ 0. 56. 56. inf 77. 45. 69. 49. 47. 49.]
(100, 100)
